In [1]:
import pandas as pd

In [3]:
#Carreguem l'arxiu a inspeccionar
af_desc_df = pd.read_csv("../Data_cleaning/selectedVars/aforaments-descriptiu.csv")

In [5]:
#Busca els valors nuls
nulls = af_desc_df.isnull().sum()
print(nulls)

Codi_Barri      1930
Codi_barri      3097
Id_aforament       3
dtype: int64


Observem dues columnes per al codi del barri, amb gran nombre de valors nuls. Comprovem si la suma dels nuls de tots dos és el total de registres, ja que això indicaria que hi ha alguna errada en combinar els arxius csv que fa que la columna s'hagi duplicat i tingui el valor nul en una d'elles quan l'altre està informada.

In [8]:
print("Total nuls columnes codi_barri: ", af_desc_df['Codi_Barri'].isnull().sum()+af_desc_df['Codi_barri'].isnull().sum())
print("Llargada dataset: ", len(af_desc_df))

Total nuls columnes codi_barri:  5027
Llargada dataset:  5022


Observem que ha d'haver-hi alguna columna on totes dues columnes siguin nul·les. Comprovem si coincideixen amb aquelles on Id_aforament també ho són

In [11]:
#Seleccionem només els registres on Id_aforament no és nul. A més, aquest registre no ens interessa, donat que no podrem identificar 
#el dispositiu de mesura en no tenir número d'identificador. 
af_desc_filtered = af_desc_df[af_desc_df['Id_aforament'].notnull()]

#Tornem a comprovar les llargades dels nuls codis barri amb el total del dataset
print("Total nuls columnes codi_barri: ", af_desc_filtered['Codi_Barri'].isnull().sum()+af_desc_filtered['Codi_barri'].isnull().sum())
print("Llargada dataset: ", len(af_desc_filtered))

Total nuls columnes codi_barri:  5021
Llargada dataset:  5019


Continua havent més nuls, per tant hi ha algun registre on totes dues columnes son nul·les. A continuació, substituim els nuls de la columna Codi_Barri per el valor de la columna Codi_barri, i eliminem aquesta darrera. Després comprovem si queden valors nuls a la columna Codi_Barri

In [14]:
af_desc_filtered['Codi_Barri'] = af_desc_filtered['Codi_Barri'].fillna(af_desc_filtered['Codi_barri'])
af_desc_filtered = af_desc_filtered.drop('Codi_barri', axis=1)
print("Total nuls columnes codi_barri: ", af_desc_filtered.isnull().sum())

Total nuls columnes codi_barri:  Codi_Barri      2
Id_aforament    0
dtype: int64


C:\Users\Usuario\AppData\Local\Temp\ipykernel_10112\1439522241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  af_desc_filtered['Codi_Barri'] = af_desc_filtered['Codi_Barri'].fillna(af_desc_filtered['Codi_barri'])


In [16]:
#Finalment, eliminem els dos registres amb valors nuls que queden, ja què suposen una pèrdua irrellevant de dades.
af_desc_nonull = af_desc_filtered[af_desc_filtered['Codi_Barri'].notnull()]
af_desc_nonull.to_csv("../Data_cleaning/selectedVars/aforaments-descriptiu.csv", index=False)

El tractament dels diferents nuls dependrà del que extraiem de l'anàlisi dels valors nuls. Ara començarem amb la càrrega del següent arxiu

In [19]:
#Carreguem l'arxiu a inspeccionar
af_det_df = pd.read_csv("../Data_cleaning/selectedVars/aforaments-detall.csv")
#Busca els valors nuls
nulls = af_det_df.isnull().sum()
print(nulls)

Id_aforament    2
Mes             2
Any             2
Valor_IMD       2
dtype: int64


Únicament trobem dos nuls a cada columna. Comprovem si es tracta dels mateixos registres.

In [22]:
af_det_nulls = af_det_df[af_det_df.isnull().any(axis=1)]
af_det_nulls

,Id_aforament,Mes,Any,Valor_IMD
43691,NaN,NaN,NaN,NaN
97840,NaN,NaN,NaN,NaN


Efectivament, es tracta de dues fileres amb tots els valors nuls, i que per tant, directament eliminarem donat que no afectarà al nostre estudi.

In [25]:
#Eliminem fileres amb nuls
af_det_cl = af_det_df.dropna()
#Substituim el csv amb aquest net
af_det_cl.to_csv("../Data_cleaning/selectedVars/aforaments-detall.csv", index=False)

Carreguem ara el següent, 'cens-activitats-comercials'

In [28]:
#Carreguem l'arxiu a inspeccionar
cac_df = pd.read_csv("../Data_cleaning/selectedVars/cens-activitats-comercials.csv")
#Busca els valors nuls
nulls = cac_df.isnull().sum()
print(nulls)

Codi_Barri              50000
Nom_Barri               50000
CBARRI                 100007
NBARRI                 100007
Data_Revisio           100050
DATA                   100000
Codi_Grup_Activitat    100000
Nom_Grup_Activitat     100000
dtype: int64


C:\Users\Usuario\AppData\Local\Temp\ipykernel_10112\1529090729.py:2: DtypeWarning: Columns (1,3,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  cac_df = pd.read_csv("../Data_cleaning/selectedVars/cens-activitats-comercials.csv")


En primer lloc, existeix un altre dataset amb els codis de barri i noms de barri. A partir de les columnes Nom_Barri i NBARRI podem assignar gran part dels Codi_Barri. El carreguem


In [31]:
barris_df = pd.read_csv("../Data_cleaning/selectedVars/unitats-administratives-barris.csv")
#Creem un diccionari amb els codis de barri i noms de barri
code_barris = dict(zip(barris_df['nom_barri'], barris_df['codi_barri']))
#Assignem aquells que sigui el Codi_Barri nul a partir de la columna CBARRI
cac_df.loc[cac_df['Codi_Barri'].isnull(), 'Codi_Barri'] = cac_df['Nom_Barri'].map(code_barris)
#Fem el mateix amb la columna NBARRI
cac_df.loc[cac_df['Codi_Barri'].isnull(), 'Codi_Barri'] = cac_df['NBARRI'].map(code_barris)
print(cac_df.isnull().sum())

Codi_Barri              10836
Nom_Barri               50000
CBARRI                 100007
NBARRI                 100007
Data_Revisio           100050
DATA                   100000
Codi_Grup_Activitat    100000
Nom_Grup_Activitat     100000
dtype: int64


Hem reduit significativament el nombre de registres sense codi de barri. Mirem ara d'assignar als nuls Codi_Barri, la columna CBARRI que pot indicar el mateix

In [34]:
cac_df['Codi_Barri'] = cac_df['Codi_Barri'].fillna(cac_df['CBARRI'])
print(cac_df.isnull().sum())

Codi_Barri                  7
Nom_Barri               50000
CBARRI                 100007
NBARRI                 100007
Data_Revisio           100050
DATA                   100000
Codi_Grup_Activitat    100000
Nom_Grup_Activitat     100000
dtype: int64


Ara mirarem de fer el mateix amb el camp Data_Revisio, assignant-li el camp DATA

In [37]:
cac_df['Data_Revisio'] = cac_df['Data_Revisio'].fillna(cac_df['DATA'])
print(cac_df.isnull().sum())

Codi_Barri                  7
Nom_Barri               50000
CBARRI                 100007
NBARRI                 100007
Data_Revisio            50050
DATA                   100000
Codi_Grup_Activitat    100000
Nom_Grup_Activitat     100000
dtype: int64


Tot i que encara el nombre de valors nuls és molt elevat (1/3 part del dataset no té data de revisió informada), aquests registres no ens interessen ja què no els podem assignar a cap any. Els eliminem, ja què a més hem de tenir en compte que aquest dataset el combinarem amb un altre amb informació similar d'altres anys.

In [40]:
cac_df2 = pd.read_csv("../Data_cleaning/selectedVars/cens-locals-planta-baixa-act-economica.csv")
#Calculem quin seria el percentatge respecte el total dels dos datasets si es combinessin
perc_nuls = ((cac_df['Data_Revisio'].isnull().sum())/(len(cac_df)+len(cac_df2)))*100
print("El percentatge de registres sense identificar la data sobre el total és del ", perc_nuls, "%")

El percentatge de registres sense identificar la data sobre el total és del  25.025 %


El percentatge continua essent força elevat, però si mirem la longitud del dataset, veiem que hi ha suficients registres per analitzar eliminant els que no tenen barri.

In [43]:
print(len(cac_df)+len(cac_df2)-cac_df['Data_Revisio'].isnull().sum())

149950


In [45]:
cac_df = cac_df[cac_df['Data_Revisio'].notnull()]
cac_df = cac_df[cac_df['Codi_Barri'].notnull()]
len(cac_df)

99950

Veiem que entre els dos datasets continuem tenint prop de 100000 registres, que són més que suficients. Pel que fa als nuls en la columna 'Codi_Grup_Activitat' no els tractarem i els deixarem tal qual. A l'hora de calcular les variables necessàries, els tractarem com si fossin activitats no típiques dels processos de gentrificació/turistificació.

In [48]:
cac_df.to_csv("../Data_cleaning/selectedVars/cens-activitats-comercials.csv", index=False)

Carreguem el següent dataset, 'culturailleure-bibliotequesimuseus'

In [51]:
#Carreguem l'arxiu a inspeccionar
bim_df = pd.read_csv("../Data_cleaning/selectedVars/culturailleure-bibliotequesimuseus.csv")
#Busca els valors nuls
nulls = bim_df.isnull().sum()
print(nulls)

addresses_neighborhood_id      3
addresses_neighborhood_name    3
secondary_filters_name         0
created                        0
modified                       0
dtype: int64


In [53]:
#Calculem percentatge de nuls
perc_nuls = (bim_df['addresses_neighborhood_id'].isnull().sum()/len(bim_df))*100
print("El percentatge de nuls és de ", perc_nuls, "%")

El percentatge de nuls és de  0.6637168141592921 %


Novament el nombre de nuls és mínim respecte el total de registres. Per tant, eliminem els registres nuls i guardem el csv net

In [56]:
bim_clean = bim_df.dropna()
bim_clean.to_csv("../Data_cleaning/selectedVars/culturailleure-bibliotequesimuseus.csv", index=False)

Els següents tres datasets són similars, donat que estan relacionats amb l'assistència a esdeveniments culturals. Carreguem els tres.

In [59]:
#Carreguem l'arxiu a inspeccionar
dae_df = pd.read_csv("../Data_cleaning/selectedVars/dades-arts-esceniques.csv")
daf_df = pd.read_csv("../Data_cleaning/selectedVars/dades-festivals.csv")
dme_df = pd.read_csv("../Data_cleaning/selectedVars/dades-museus-exposicions.csv")

In [61]:
#Busca els valors nuls en cada dataset
nulls_dae = dae_df.isnull().sum()
nulls_daf = daf_df.isnull().sum()
nulls_dme = dme_df.isnull().sum()
print("Nulls dades arts escèniques: ", nulls_dae)
print("Nulls dades festivals: ", nulls_daf)
print("Nulls dades museus: ", nulls_dme)

Nulls dades arts escèniques:  Codi_Barri           204
Nom_Barri            204
Any                    0
Tipus_Equipament    3143
Indicador              0
Valor                  0
dtype: int64
Nulls dades festivals:  Espai                           1469
Any                                0
Assistents_fora_de_Barcelona    2473
Assistents_a_Barcelona          1981
Assistents                      1277
dtype: int64
Nulls dades museus:  Codi_Barri          52
Nom_Barri           52
Any                  0
TipusEquipament    313
Indicador            0
Valor                0
dtype: int64


Hi ha dos datasets amb un nombre baix de nuls en codi barri, i que coincideix amb nombre amb nom_barri. Si fossin els mateixos registres, els eliminarem, ja que no podrem identificar-los

In [64]:
nulls_arts = dae_df[dae_df['Codi_Barri'].isnull()]
nulls_museus = dme_df[dme_df['Codi_Barri'].isnull()]
print(nulls_arts.isnull().sum(), "total registres: ", len(nulls_arts))
print(nulls_museus.isnull().sum(), "total registres: ", len(nulls_museus))

Codi_Barri          204
Nom_Barri           204
Any                   0
Tipus_Equipament    189
Indicador             0
Valor                 0
dtype: int64 total registres:  204
Codi_Barri         52
Nom_Barri          52
Any                 0
TipusEquipament     0
Indicador           0
Valor               0
dtype: int64 total registres:  52


Podem eliminar aquests registres.

In [67]:
dae_df = dae_df[dae_df['Codi_Barri'].notnull()]
dme_df = dme_df[dme_df['Codi_Barri'].notnull()]

In [69]:
#Mirem els valors únics de les columnes amb nulls en aquests dos datasets
print("Valors únics 'Tipus_Equipament' arts escèniques: ", dae_df['Tipus_Equipament'].unique())
print("Valors únics 'TipusEquipament' museus: ", dme_df['TipusEquipament'].unique())

Valors únics 'Tipus_Equipament' arts escèniques:  ["Sales d'arts escèniques" 'Cinemes' 'Fàbriques de Creació'
 'Grans auditoris' nan 'Museus i col·leccions']
Valors únics 'TipusEquipament' museus:  [nan 'Arxius' "Espais d'interès patrimonial" "Centres d'exposicions"
 'Museus i col·leccions' "Espais d'interès arquitectònic"
 "Sales d'arts escèniques"]


Donat que aquesta columna no ens dona cap informació rellevant pel tractament de les dades, li assignarem als valors nuls el valor 'Arts escèniques' en el cas del primer dataset, i 'Museus i exposicions' en el cas del segon. Guardem aquests dos nous csv substituint els originals.

In [72]:
dae_df['Tipus_Equipament'] = dae_df['Tipus_Equipament'].fillna("Arts escèniques")
dme_df['TipusEquipament'] = dme_df['TipusEquipament'].fillna("Museus i exposicions")
dae_df.to_csv("../Data_cleaning/selectedVars/dades-arts-esceniques.csv", index=False)
dme_df.to_csv("../Data_cleaning/selectedVars/dades-museus-exposicions.csv", index=False)

En el cas de dades a festivals, primer mirem els valors únics de la columna 'Espai'

In [75]:
daf_df['Espai'].unique()

array(['Diversos espais de la ciutat', 'SAT! Sant Andreu Teatre',
       'Sala Heliogàbal i Casinet d¿Hostafrancs',
       'Fira de Barcelona Gran Via',
       'Edifici Històric de la Universitat de Barcelona',
       'Barri de Poblenou', 'Ateneu Popular 9 Barris',
       'Xarxa de centres cívics', "L'Auditori",
       'Antiga Fàbrica Estrella Damm', 'Poble Espanyol',
       'Institut Francés de Barcelona', 'Disseny Hub Barcelona',
       'Centre de Cultura Contemporània de Barcelona. CCCB',
       'Fira de Barcelona Montjuïc', 'Parc del Fòrum',
       'Parc de les Glòries', 'Districte de Ciutat Vella',
       'Fabra i Coats Fàbrica de creació', 'Castell de Montjuïc',
       'La Pedrera', 'Fossat del Mercat de Sant Antoni',
       'Poble Espanyol ', 'Districte de les Corts',
       'Moll de la Fusta de Barcelona', 'Jardins del Palau de Pedralbes',
       "Districte de l'Eixample", 'Centre Cultural la Farinera',
       'Palau Güell', 'Museu Can Framis', 'Barri Gòtic',
       'Pavelló Mi

En aquest cas, la columna 'Espai' ens indica el lloc on es produeix l'event, i a partir del dataset 'equipaments-culturals.icub' podrem identificar el barri on es produeix. Per tant, en aquest cas els valors nuls no ens aporten cap informació i eliminarem els registres.

In [78]:
daf_df = daf_df[daf_df['Espai'].notnull()]

Pel que fa a les altres columnes que presenten nuls, fan referència al nombre d'assistents de fora de Barcelona, de Barcelona i el total. En aquest cas únicament ens interessa la columna d'Assistents totals. Mirem si a partir de les altres dues es poden omplir nuls sumant assistents de fora de Barcelona i de Barcelona.

In [81]:
daf_df['Assistents'] = daf_df['Assistents'].fillna(daf_df['Assistents_fora_de_Barcelona']+daf_df['Assistents_a_Barcelona'])
print(daf_df.isnull().sum())

Espai                              0
Any                                0
Assistents_fora_de_Barcelona    1006
Assistents_a_Barcelona           523
Assistents                       750
dtype: int64


Amb aquesta assignació només hem pogut assignar als valors nuls d'Assistents, aquells en que no hi havia cap nul a les altres dues columnes. Però encara podem reassignar valors, els valors no nuls de les columnes d'assistència a Barcelona, o fora de Barcelona, a Assistents quan aquesta és nul·la

In [84]:
daf_df['Assistents'] = (daf_df['Assistents'].fillna(daf_df['Assistents_fora_de_Barcelona']))
daf_df['Assistents'] = (daf_df['Assistents'].fillna(daf_df['Assistents_a_Barcelona']))
print(daf_df.isnull().sum())

Espai                              0
Any                                0
Assistents_fora_de_Barcelona    1006
Assistents_a_Barcelona           523
Assistents                       261
dtype: int64


Eliminem les dues columnes sobrants, i els nuls de la columna Assistents, donat que no ens donen informació.

In [87]:
columnes = ['Assistents_fora_de_Barcelona', 'Assistents_a_Barcelona']
daf_df = daf_df.drop(columnes, axis=1)
daf_df = daf_df.dropna()

Finalment, substituim el csv original

In [90]:
daf_df.to_csv("../Data_cleaning/selectedVars/dades-festivals.csv", index=False)

Ara carreguem l'arxiu d'equipaments-restaurants

In [93]:
#Carreguem l'arxiu a inspeccionar
rest_df = pd.read_csv("../Data_cleaning/selectedVars/equipament-restaurants.csv")
#Imprimim els valors nuls per variable
print(rest_df.isnull().sum())

addresses_neighborhood_id      0
addresses_neighborhood_name    1
secondary_filters_name         0
created                        0
modified                       0
dtype: int64


Únicament trobem un registre nul, corresponent al nom del barri, però com tenim l'identificador, no caldrà fer cap acció sobre el valor nul.

Ara carreguem l'arxiu d'habitatges d'ús turístic

In [130]:
#Carreguem l'arxiu a inspeccionar
hut_df = pd.read_csv("../Data_cleaning/selectedVars/habitatges-us-turistic.csv")
#Imprimim els valors nuls per variable
print(hut_df.isnull().sum())

CODI_BARRI    42304
BARRI          4247
Any               0
dtype: int64


In [132]:
#Mirem el percentatge que suposen, donat que sense l'identificador del barri no podem treballar, els haurem d'eliminar de totes maneres.
print("El percentatge de nuls és de :", (hut_df['CODI_BARRI'].isnull().sum()/len(hut_df))*100, "%")

El percentatge de nuls és de : 68.6496924849488 %


Farem servir un altre cop el dataset amb els codis de barri i noms de barri. A partir de la columna BARRI podem assignar gran part dels CODI_BARRI. El carreguem

Assignem els codis del barri a partir del dataframe en aquells en que CODI_BARRI és nul

In [136]:
#Creem un diccionari amb els codis de barri i noms de barri
code_barris = dict(zip(barris_df['nom_barri'], barris_df['codi_barri']))
#Assignem aquells que sigui el CODI_BARRI nul a partir de la columna BARRI
hut_df.loc[hut_df['CODI_BARRI'].isnull(), 'CODI_BARRI'] = hut_df['BARRI'].map(code_barris)
print(hut_df.isnull().sum())

CODI_BARRI    6946
BARRI         4247
Any              0
dtype: int64


Detectem que hi ha encara codis sense assignar. Es pot tractar perquè el nom del barri no estigui introduit de la mateixa manera que al dataset dels codis. Comparem els valors únics d'un i altre.

In [139]:
noms_barris_oficial = list(barris_df['nom_barri'].unique())
noms_barris_hut = list(hut_df['BARRI'].unique())

Comparem les llistes de valors únics per extreure quins són els diferents.

In [143]:
barri_dif = []
for i in noms_barris_hut:
    if i not in noms_barris_oficial:
        barri_dif.append(i)
print(barri_dif)

['el Poble Sec', nan, 'Sant Pere Santa Caterina i la Ribera', 'Vallvidrera el Tibidabo i les Planes']


Si comparem amb els noms oficials, tenim les següents correspondències (el segon és l'oficial):

el Poble Sec: el Poble-sec

Sant Pere Santa Caterina i la Ribera: Sant Pere, Santa Caterina i la Ribera

Vallvidrera el Tibidabo i les Planes: Vallvidrera, el Tibidabo i les Planes

Per tant, fem les substitucions que corresponguin al dataset i tornem a fer l'assignació de codis als valors nuls

In [146]:
hut_df.loc[hut_df['BARRI']=='el Poble Sec', 'BARRI'] = 'el Poble-sec'
hut_df.loc[hut_df['BARRI']=='Sant Pere Santa Caterina i la Ribera', 'BARRI'] = 'Sant Pere, Santa Caterina i la Ribera'
hut_df.loc[hut_df['BARRI']=='Vallvidrera el Tibidabo i les Planes', 'BARRI'] = 'Vallvidrera, el Tibidabo i les Planes'

hut_df.loc[hut_df['CODI_BARRI'].isnull(), 'CODI_BARRI'] = hut_df['BARRI'].map(code_barris)

print(hut_df.isnull().sum())

CODI_BARRI    4241
BARRI         4247
Any              0
dtype: int64


Ara comprovarem que tots els registres amb CODI_BARRI nul ja no es poden deduir des d'altres camps

In [149]:
huts_null = hut_df[hut_df['CODI_BARRI'].isnull()]
print(huts_null.isnull().sum())

CODI_BARRI    4241
BARRI         4241
Any              0
dtype: int64


Com podem comprovar, tots els registres son nuls. Per tant, els eliminem, i guardem els canvis en el csv original

In [152]:
hut_df = hut_df.dropna()
hut_df.to_csv("../Data_cleaning/selectedVars/habitatges-us-turistic.csv", index=False)

Importem el darrer dataset on tenim valors nuls

In [122]:
#Carreguem l'arxiu a inspeccionar
ter_df = pd.read_csv("../Data_cleaning/selectedVars/terrasses-comercos-vigents.csv")
#Imprimim els valors nuls per variable
print(ter_df.isnull().sum())

CODI_BARRI    16
NOM_BARRI     16
DATA_EXPLO     0
TAULES         0
dtype: int64


Novament, el nombre de nuls a CODI_BARRI i NOM_BARRI coincideix. Si es tracta dels mateixos registres, els eliminarem

In [125]:
ter_nuls = ter_df[ter_df['CODI_BARRI'].isnull()]
print("Total registres: ", len(ter_nuls), "Nuls detectats: ", ter_nuls.isnull().sum())

Total registres:  16 Nuls detectats:  CODI_BARRI    16
NOM_BARRI     16
DATA_EXPLO     0
TAULES         0
dtype: int64


Eliminem els nuls i substituim el csv original

In [128]:
ter_df = ter_df.dropna()
ter_df.to_csv("../Data_cleaning/selectedVars/terrasses-comercos-vigents.csv", index=False)